In [ ]:
# new data structure for STOPWATCHER
# build up tables BY VEHICLE - a history of observations of each vehicle
# then can go and reconstruct unique history for each VEHICLE, based on last observeration before it STOPPED APPROACHING each stop in the _stoplist_

In [2]:
# relative import
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from src.buses.reportcard_helpers import *

In [3]:
import time
import pandas as pd

In [4]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print '%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000)
        return result
    return timed

In [5]:
source = 'nj'
route = 119

In [6]:
(conn, db) = db_setup(route)

In [7]:
# only get buses APPROACHING A STOP
# arrival_query = ('SELECT * FROM stop_predictions \
#                WHERE (rd = %s AND pt = "APPROACHING") \
#                ORDER BY timestamp;' % route)
arrival_query = ('SELECT * FROM stop_predictions;')

In [8]:
df = pd.read_sql_query(arrival_query, conn)
df

,pkey,cars,consist,fd,m,name,pt,rd,rn,scheduled,stop_id,stop_name,v,timestamp
0,1,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,8,119,119,false,26229,PORT AUTHORITY BUS TERMINAL,6200,2018-06-07 14:14:04.349723
1,2,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,16,119,119,false,21854,WILLOW AVE + 19TH ST,6200,2018-06-07 14:14:04.471519
2,3,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,17,119,119,false,20523,WILLOW AVE + 15TH ST,6200,2018-06-07 14:14:04.521519
3,4,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,22,119,119,false,20668,CONGRESS ST + PALISADE AVE,6200,2018-06-07 14:14:04.569904
4,5,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,22,119,119,false,20670,CONGRESS ST + WEBSTER AVE,6200,2018-06-07 14:14:04.620028
5,6,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,23,119,119,false,20671,CONGRESS ST + HANCOCK AVE,6200,2018-06-07 14:14:04.670139
6,7,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,24,119,119,false,20672,CONGRESS ST + CENTRAL AVE,6200,2018-06-07 14:14:04.719928
7,8,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,25,119,119,false,20629,CENTRAL AVE + SOUTH ST,6200,2018-06-07 14:14:04.763751
8,9,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,25,119,119,false,20630,CENTRAL AVE + THORNE ST,6200,2018-06-07 14:14:04.817999
9,10,,,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,1,StopPrediction,26,119,119,false,20631,CENTRAL AVE + BOWERS ST,6200,2018-06-07 14:14:04.865516


In [ ]:
df = timestamp_fix(df)
df.head(5)

In [ ]:
# 1 group all arrival predictions by unique vehicle-stop combinations
df.groupby(['v','stop_id'])

# 2 gets the keys for unique vehicle-stop combinations
# a list of tuples
# [(u'5403', u'20640'), (u'5704', u'20933'), .....

keys = df.groupby(['v','stop_id']).groups.keys()


In [ ]:
@timeit
def make_history(df):

    # THIS IS WAYYYYYYY FASTER !!!

    # history data structure = assign every row in df to a dict keyed to unique vehicle/stop_id instance
    # {
    #  5403_20640: [row1, row15, ...]
    #  5704_20933: [row2, row37, ...]    
    # }

    from collections import defaultdict
    history = defaultdict(list)

    for index,row in df.iterrows():
        key = row['v'] + '_' + row['stop_id']
        history[key].append(row)

    # sort them and slice them

    keepers = defaultdict(list)
    discards = defaultdict(list)

    # iterate over the dict
    for key_copy, arrivals in history.iteritems():

        # sort each arrival list
        arrivals.sort(key=lambda x: x.timestamp)

        #
        #
        # NEED TO GROUPBY ONES THAT HAVE THE SAME GENERAL ARRIVAL WINDOW
        #
        # PROBLEM WITH BELOW IS THAT IT DOESNT WORK WHEN WE HAVE A FULL HISTORY
        # BECAUSE IT DOESNT DETECT EVENTS? e.g. VEHICLE-STOP COMBINATIONS ARE NOT UNIQUE FOR MORE THAN A FEW HOURS
        #
        #   

        # put last row in keepers
        keepers[key_copy].append(arrivals[-1])

        # put everything else in discards
        discards[key_copy].append(arrivals[:-1])
        
        return keepers,discards


In [ ]:
# this isnt working now.... only returning 1?

In [ ]:
(keepers,discards) = make_history(df)


In [ ]:
print keepers

In [ ]:
print discards

In [ ]:
# 1. NEW STOPWATCHER V1 - single stop frequency of service

# use the new stopwatcher data model and drill down to an appraoches table for unique (stop,route) listing all APPROACHING observations (an 'approach') in history. then either excerpt a separate table or set a flag on the last approach in each sequence. that timestamp is then treated as the actual arrival time.

# then use that table of arrivals to calculate report card stats and visualizations

# ***THIS IS KIND OF A HACKY WAY OF LOCALIZING THE BUS TO THE STOP (but it works, letting CleverDevices do the heavy lifting of interpolation)
